<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Felipe Aguilar
- Nombre de alumno 2: Joaquín Pérez O.


### **Link de repositorio de GitHub:** `https://github.com/JoaquinPerez17/Lab-MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime as dt
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Laboratorios MDS/Laboratorio_6/online_retail_II_cleaned.pickle'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [ ]:
df_retail = pd.read_pickle(path)
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def custom_features(dataframe_in):
    groups = dataframe_in.groupby('Customer ID')

    # Length
    Length = pd.DataFrame(groups.agg(['first', 'last'])['InvoiceDate'].reset_index())
    Length['Length'] = (Length['last'] - Length['first']).dt.days
    Length = Length.loc[:, ['Customer ID', 'Length']]

    # Recency
    Recency_fecha = pd.DataFrame(groups.agg(['last'])['InvoiceDate'].reset_index())
    Recency_fecha.rename(columns={"last": "fecha final"}, inplace = True)
    Recency_fecha['fecha final'] = Recency_fecha['fecha final'] + dt.timedelta(days=1)
    Recency = pd.merge(left=df_retail, right=Recency_fecha, on='Customer ID', how='left')
    Recency['Recency'] = (Recency['fecha final'] - Recency['InvoiceDate']).dt.days
    Recency = Recency.groupby('Customer ID').agg('mean')['Recency'].reset_index()

    # frecuency
    Frecuency = pd.DataFrame(groups['Invoice'].agg(['unique']).reset_index())
    Frecuency.rename(columns={"unique": "Frecuency"}, inplace = True)
    Frecuency['Frecuency'] = Frecuency['Frecuency'].apply(lambda x: len(x.astype(int)))

    # Monetary
    Monetary = dataframe_in.copy()
    Monetary['Gasto'] = Monetary['Quantity'] * Monetary['Price']
    Monetary = Monetary.loc[:, ['Invoice', 'Customer ID', 'Gasto']]
    Monetary = pd.DataFrame(Monetary.groupby('Customer ID').agg('sum').reset_index())
    Monetary['Monetary'] = Monetary['Gasto']/Frecuency['Frecuency']
    Monetary = Monetary.loc[:, ['Customer ID', 'Monetary']]

    # Periodicy
    Periodicity = dataframe_in.copy()
    Periodicity = Periodicity.groupby('Invoice').nth(0).reset_index() 
    Periodicity['Periodicity'] = Periodicity.groupby('Customer ID', group_keys=False).apply(lambda x: (x['InvoiceDate'] - x['InvoiceDate'].shift(1)).dt.days)
    Periodicity = Periodicity.groupby('Customer ID').std()['Periodicity'].reset_index()
    Periodicity['Periodicity']= Periodicity['Periodicity'].fillna(0)


    df_out = pd.merge(Length, Recency, on='Customer ID', how='left')
    df_out = pd.merge(df_out, Frecuency, on='Customer ID', how='left')
    df_out = pd.merge(df_out, Monetary, on='Customer ID', how='left')
    df_out = pd.merge(df_out, Periodicity, on='Customer ID', how='left')

    return df_out

In [ ]:
df_cf = custom_features(df_retail)

In [ ]:
display(df_cf)

,Customer ID,Length,Recency,Frecuency,Monetary,Periodicity
0,12346.0,196,68.696970,11,33.896364,36.659999
1,12347.0,37,21.845070,2,661.660000,0.000000
2,12348.0,0,1.000000,1,222.160000,0.000000
3,12349.0,181,84.215686,3,890.380000,101.823376
4,12351.0,0,1.000000,1,300.930000,0.000000
...,...,...,...,...,...,...
4309,18283.0,275,158.649770,6,103.228333,61.832839
4310,18284.0,0,1.000000,1,461.680000,0.000000
4311,18285.0,0,1.000000,1,427.000000,0.000000
4312,18286.0,247,63.671642,2,648.215000,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return (X - X.min(axis=0))/(X.max(axis=0) - X.min(axis=0))

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [ ]:
Features = FunctionTransformer(custom_features)
pipeline_features = Pipeline([('prepro', Features)])
pipeline_scaler = Pipeline([('scaler', MinMax())])
transformer = ColumnTransformer([('Transform', pipeline_scaler, ['Length', 'Recency', 'Frecuency', 'Monetary', 'Periodicity'])])
tsne = TSNE(n_components=2, random_state=42)

pipeline_LRMFP = Pipeline([('Preprocessing', pipeline_features), ('Transformer', transformer), ('T-SNE', tsne)])

In [ ]:
retail_features_tsne = pipeline_LRMFP.fit_transform(df_retail)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
fig = px.scatter(x=retail_features_tsne[:, 0], y=retail_features_tsne[:, 1])
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [ ]:
Features = FunctionTransformer(custom_features)
pipeline_features = Pipeline([('prepro', Features)])
pipeline_scaler = Pipeline([('scaler', MinMax())])
transformer = ColumnTransformer([('Transform', pipeline_scaler, ['Length', 'Recency', 'Frecuency', 'Monetary', 'Periodicity'])])

pipeline_scaled_LRMFP = Pipeline([('Preprocessing', pipeline_features), ('Transformer', transformer)])
retail_scaled = pipeline_scaled_LRMFP.fit_transform(df_retail)

In [ ]:
intertias = [
    [i, KMeans(n_clusters=i, random_state=42).fit(retail_scaled).inertia_]
    for i in range(2, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

,n° clusters,inertia
0,2,206.245125
1,3,127.915093
2,4,102.089336
3,5,81.488313
4,6,72.390731
5,7,64.613773
6,8,59.936521
7,9,56.222687
8,10,52.744124
9,11,49.461692


In [ ]:
px.line(
    intertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)

Según lo que se observa en el gráfico, los cambios drásticos dejan de ocurrir a partir del número de clusters 8, de esta forma, el número de clusters óptimos serían justamente 8. Si se mirara con más detalle el gráfico, probablemente se podría considerar que el número de clusters óptimos sería 12 o incluso 14, aunque no queda muy claro a partir de la visualización anterior. Sin embargo, 8 también es un número válido pues a partir de ese punto el error ya ha disminuido y la pendiente es mucho menor.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [ ]:
kmeans = KMeans(n_clusters=8)
kmeans.fit(retail_scaled)

KMeans()

In [ ]:
df_cf['Cluster'] = kmeans.labels_
df_LRMFP = pd.DataFrame(retail_scaled)
df_LRMFP['Cluster'] = kmeans.labels_

In [ ]:
df_cf.groupby('Cluster').mean()

,Length,Recency,Frecuency,Monetary,Periodicity
Cluster,,,,,
0,3.188242,2.855581,1.264556,359.184158,0.218186
1,249.169922,128.169906,6.142578,395.398735,24.685333
2,191.210300,85.953412,3.759657,337.298720,84.164767
3,313.231579,150.509741,3.652632,336.512228,152.518710
4,78.426087,41.215879,3.156522,382.078174,12.009267
5,310.285106,131.131107,5.523404,384.114655,69.673906
6,158.396552,79.572668,3.879310,366.673690,14.125464
7,343.935897,176.815005,14.761905,436.560908,26.891181


A partir de la tabla anterior, se observa que sí es posible observar agrupaciones coherentes. Por ejemplo, se observa que en Length existe una clara separación entre grupos que poseen una media alto versus los que poseen una media baja. Los grupos que poseen un Length similar, se distancian notoriamente en la variable Periodicity. Además, se puede observar que la variable Recency también ayuda a distanciar entre grupos.

Con respecto al tipo de cliente


*  Hay 4 grupos en los que existen clientes más fieles (**clusters 1, 3, 5, 7**), lo que se refleja a partir de la variable Length, pero dentro de esos 4 hay 2 grupos que representan aún más fidelidad a la tienda pues tienen un Periodicity alto. Y dentro de estos dos últimos, hay un grupo que es incluso más asiduo a la tienda pues tiene una alta Frecuency (**cluster 7** ).
*   El **cluster 0** representa el tipo de clientes que ha ido muy poco a realizar compras, y puede reflejar visitas puntuales.
*  Hay una serie de clientes que podrían ser clientes más nuevos, que son fieles a la tienda a pesar de sus pocas visitar, pero aún no han ido lo suficiente como para poder destacar en Recency.





**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [ ]:
df_LRMFP = pd.DataFrame(retail_features_tsne)
df_LRMFP['Cluster'] = kmeans.labels_

In [ ]:
px.scatter(
    df_LRMFP,
    x=0,
    y=1,
    color='Cluster',
)

Dentro de la visualización es posbile observar que los grupos logran separarse de manera correcta, pues efectivamente se van marcando límites entre clusters dentro del plano. El cluster 0 es el que presenta la forma más extraña. Si volvemos a la tabla del cluster 0, se observa que es el que tiene los valores más distanciados de todos los otros clusters, siendo Periodicity la varible que más marca la diferencia. Como los valores NaN fueron transformados a 0 en esa variable, se pudo haber formado ese cluster, que si bien es cierto se puede identificar como grupo pues se distancia de los demás, la forma es bastante extraña.

A pesar de identificar cluster dentro del gráfico, este es un espacio transformado a partir de más dimensiones, por lo que no podemos concluir porqué los grupos se ubican en esas determinadas posiciones.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>